In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from transformers import pipeline, T5Tokenizer, T5EncoderModel
import torch
import re
import json
import numpy as np

## Negative Samples Creation w/ Embeddings and Coisne Similarity

Here we chose the approach to create embeddings from the prot_t5_xl model and then check if the embedding is in a certain "un-similarity range" which is a self-defined threshold. This becuase evaluations showed that this is (at least as far as we consider) the "best" approach.

In [ ]:
read_path_beta = "../../data/customDatasets/paired_concatenated.tsv"
paired_df = pd.read_csv(read_path_beta, sep="\t")

In [ ]:
paired_df

In [ ]:
len(set(paired_df["Epitope"]))

In [ ]:
epitope_counts = paired_df['Epitope'].value_counts().reset_index()
print(epitope_counts)
epitope_counts.columns = ['Epitope Name', 'Count'] 

plt.figure(figsize=(10, 6))
plt.hist(paired_df['Epitope'], bins=len(epitope_counts), edgecolor='k', alpha=0.7)
plt.xlabel('Epitope Value')
plt.ylabel('Frequency')
plt.title('Distribution of Epitope Data')
plt.xticks([])  # removes the x-axis labels
plt.show()

In [ ]:

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Using device: {}".format(device))


In [ ]:

#@title Load encoder-part of ProtT5 in half-precision. { display-mode: "form" }
# Load ProtT5 in half-precision (more specifically: the encoder-part of ProtT5-XL-U50 in half-precision)
transformer_link = "Rostlab/prot_t5_xl_half_uniref50-enc"
print("Loading: {}".format(transformer_link))
model = T5EncoderModel.from_pretrained(transformer_link)
if device==torch.device("cpu"):
  print("Casting model to full precision for running on CPU ...")
  model.to(torch.float32) # only cast to full-precision if no GPU is available
model = model.to(device)
model = model.eval()
tokenizer = T5Tokenizer.from_pretrained(transformer_link, do_lower_case=False, legacy=True)


In [ ]:
epitopes = set(paired_df["Epitope"].to_list())

In [ ]:
# this will replace all rare/ambiguous amino acids by X and introduce white-space between all amino acids
processed_epitopes = [(sequence, " ".join(list(re.sub(r"[UZOB]", "X", sequence)))) for sequence in epitopes]
# processed_epitopes

In [ ]:
def process_batch(processed_seqs):
    # Extract just the processed sequences for tokenization
    sequences = [seq[1] for seq in processed_seqs]
    ids = tokenizer.batch_encode_plus(sequences, add_special_tokens=True, padding="longest", return_tensors="pt")
    input_ids = ids['input_ids'].to(device)
    attention_mask = ids['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    
    last_hidden_states = outputs.last_hidden_state
    
    # Now, return embeddings mapped to the original sequence
    embeddings = {}
    for i, (original_seq, _) in enumerate(processed_seqs):
        seq_len = len(original_seq)
        valid_embeddings = last_hidden_states[i,:seq_len]
        per_protein_embedding = valid_embeddings.mean(dim=0)
        # print(f"mean_embedding.shape: {per_protein_embedding.shape}")
        embeddings[original_seq] = per_protein_embedding.cpu().numpy()  # Use original sequence as key
    
    return embeddings

In [ ]:
to_path = "../../data/customDatasets/negative_samples/temp"
file_name = "Stitchr_paired_concatenated_with_epitope_embedding.tsv"

In [ ]:

batch_size = 128
sequence_to_embedding = {}

# Batch processing with a dictionary, using original sequences as keys
for i in range(0, len(processed_epitopes), batch_size):
    batch_sequences = processed_epitopes[i:i+batch_size]
    batch_embeddings = process_batch(batch_sequences)
    sequence_to_embedding.update(batch_embeddings)

    paired_df["Epitope Embedding"] = paired_df["Epitope"].map(sequence_to_embedding)

# This is needed becuase the embedding is huge and otherwise it would be stored with line breaks (\n) 
# This would make it difficult while reading the file
paired_df['Epitope Embedding'] = paired_df['Epitope Embedding'].apply(lambda x: json.dumps(x.tolist()))

paired_df.to_csv(to_path+"/"+file_name, sep="\t", index=False)



The cell above is commented out to safe time. After changing something in the underlaying dataset re-run this cell to create the up-to-date embeddings!

In [ ]:

paired_df = pd.read_csv(to_path+"/"+file_name, sep="\t")
paired_df['Epitope Embedding'] = paired_df['Epitope Embedding'].apply(lambda x: np.array(json.loads(x)))

In [ ]:
max_index = len(paired_df) - 1 
negative_epitopes_cosine = []

In [ ]:
def cosine_similarity(embedding1, embedding2): 
    cosine = np.dot(embedding1,embedding2)/(np.linalg.norm(embedding1)*np.linalg.norm(embedding2))
    return cosine

In [ ]:
def is_valid_negative(cosine_similarity, current_epitope, random_epitope): 
    is_valid = False
    cosine_min = -1
    cosine_max = 0.75

    if (cosine_similarity >= cosine_min \
        and cosine_similarity <= cosine_max) \
        and (current_epitope != random_epitope): 
        is_valid = True 

    return is_valid


In [ ]:
def search_negative_epitope_embedding(df, index, current_epitope):
    df = df
    index = index
    epitope = current_epitope
    embedding = df["Epitope Embedding"][index]
    # print(epitope_embedding)
    random_epitope_index = np.random.randint(0, max_index)
    random_epitope = df["Epitope"][random_epitope_index]
    random_epitope_embedding = df["Epitope Embedding"][random_epitope_index]
    random_mhc_a = df["MHC A"][random_epitope_index]
    random_mhc_b = df["MHC B"][random_epitope_index]
    cosine = cosine_similarity(embedding, random_epitope_embedding)

    if is_valid_negative(cosine, epitope, random_epitope): 
        negative_epitopes_cosine.append((random_epitope, random_mhc_a, random_mhc_b))
    else: 
        search_negative_epitope_embedding(df, index, current_epitope)         

In [ ]:
for i, epitope in enumerate(paired_df["Epitope"]): 
    search_negative_epitope_embedding(paired_df, i, epitope)

In [ ]:
len((negative_epitopes_cosine))

In [ ]:
epitopes = []
mhc_a = []
mhc_b = []

In [ ]:
for row_infos in negative_epitopes_cosine:
    epitopes.append(row_infos[0]) 
    mhc_a.append(row_infos[1])
    mhc_b.append(row_infos[2])

In [ ]:
negative_epitopes_cosine_dict = {"Negative Epitope": epitopes, "MHC A": mhc_a, "MHC B": mhc_b}
negative_epitopes_cosine_df = pd.DataFrame(negative_epitopes_cosine_dict)
# print(negative_epitopes_cosine_df.to_string())
negative_epitopes_cosine_df

In [ ]:
epitope_counts_negative = negative_epitopes_cosine_df['Negative Epitope'].value_counts().reset_index()
epitope_counts_negative.columns = ['Epitope Name', 'Count']
print(epitope_counts_negative) 

plt.figure(figsize=(10, 6))
plt.hist(negative_epitopes_cosine_df['Negative Epitope'].astype(str), bins=len(epitope_counts_negative), edgecolor='k', alpha=0.7)
plt.xlabel('Epitope Value')
plt.ylabel('Frequency')
plt.title('Distribution of Negative Epitope Data')
plt.xticks([])  # removes the x-axis labels
plt.show()

In [ ]:
paired_df.columns

In [ ]:
paired_with_negative_df = paired_df.drop(["Epitope Embedding", "MHC A", "MHC B"], axis=1).copy(deep=True)
paired_with_negative_df["Epitope"] = negative_epitopes_cosine
paired_with_negative_df["MHC A"] = mhc_a
paired_with_negative_df["MHC B"] = mhc_b
paired_with_negative_df["Binding"] = 0
paired_with_negative_df

In [ ]:
paired_with_negative_df = pd.concat([paired_df.drop(["Epitope Embedding"], axis=1).copy(deep=True), paired_with_negative_df], axis=0)
paired_with_negative_df["TCR_name"] = range(1, len(paired_with_negative_df)+1)
paired_with_negative_df

In [ ]:
to_path = "../../data/customDatasets/negative_samples/"
file_name = "paired_concatenated_with_negative.tsv"

In [ ]:
paired_with_negative_df.to_csv(to_path+"/"+file_name, sep="\t", index=False)